# 12 - France Football e AS

In [ ]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.action_chains import ActionChains
from time import sleep
import re
import numpy as np
import pandas as pd
from requests import get
from pandas import read_csv
from unidecode import unidecode
from bs4 import BeautifulSoup as BS

## France Football

### Qualunque anno

In [ ]:
df = read_csv('tables/nome_nomechiave.csv', encoding='utf8')
s1 = 'https://www.google.com/search?q='
s2 = '+site:francefootball.com&source=lnms&tbm=nws'
df['url_ff'] = [s1 + x.replace(' ', '+').lower() + s2 for x in df.nome_chiave]

headers = {"User-Agent":"Mozilla/5.0"}
results_ff = []

# si estrae il numero di risultati di google in corrispondenza della ricerca di ciascun giocatore sulla rivista FranceFootball
for link in df.url_ff:
    soup = BS(get(link).text,'lxml')
    stringa = soup.find('div', attrs = {'id': 'resultStats'}).text.strip().replace('.','')
    results_ff.append(int(str([int(s) for s in stringa if s.isdigit()]).strip('[]').replace(',','').replace(' ','')))

# si aggiorna il dataset df con la nuova colonna
df['ff_google_all'] = results_ff

### 2018

In [ ]:
df = read_csv('tables/nome_nomechiave.csv', encoding='utf8')
s1 = 'https://www.google.com/search?q='
s2 = '+site:francefootball.com&tbm=nws&source=lnt&tbs=qdr:y'
df['url_ff'] = [s1 + x.replace(' ', '+').lower() + s2 for x in df.nome_chiave]
df['ff_google_18'] = np.nan

# ------

driver = webdriver.Chrome(executable_path='chromedriver')
driver.maximize_window()

results = []

for i in range(300):
    if isinstance(df.ff_google_18[i], float):
        url = df.url_ff[i]
        driver.get(url)
        sleep(np.random.uniform(1))
        el = driver.find_element_by_link_text('Strumenti')
        webdriver.ActionChains(driver).move_to_element(el).click(el).perform()
        sleep(np.random.uniform(1))
        result = driver.find_element_by_id('resultStats').text.strip().replace('.','')
        result = int(str([int(s) for s in result if s.isdigit()]).strip('[]').replace(',','').replace(' ',''))
        df.loc[i, 'ff_google_18'] = result
        print((i,df.nome[i]), end=' ')
    
driver.quit()

## AS.com

### Qualunque anno

In [ ]:
s1 = 'https://www.google.com/search?q='
s2 = '+site:as.com&source=lnms&tbm=nws'
df['url_as'] = [s1 + x.replace(' ', '+').lower() + s2 for x in df.nome_chiave]

headers = {"User-Agent":"Mozilla/5.0"}
results_as = []

# si estrae il numero di risultati di google in corrispondenza della ricerca di ciascun giocatore sulla rivista AS
for link in df.url_as:
    soup = BS(get(link).text,'lxml')
    results_as.append(soup.find('div', attrs = {'id': 'resultStats'}).text.strip().split(' ')[1].replace('.',''))

# si aggiorna il dataset df con la nuova colonna
df['as_google_all'] = results_as

### 2018

In [ ]:
df = read_csv('tables/nome_nomechiave.csv', encoding='utf8')
s1 = 'https://www.google.com/search?q='
s2 = '+site:as.com&tbm=nws&source=lnt&tbs=qdr:y'
df['url_as'] = [s1 + x.replace(' ', '+').lower() + s2 for x in df.nome_chiave]
df['as_google_18'] = np.nan

# ------

driver = webdriver.Chrome(executable_path='chromedriver')
driver.maximize_window()

results = []

for i in range(300):
    if isinstance(df.as_google_18[i], float):
        url = df.url_as[i]
        driver.get(url)
        sleep(np.random.uniform(1))
        el = driver.find_element_by_link_text('Strumenti')
        webdriver.ActionChains(driver).move_to_element(el).click(el).perform()
        sleep(np.random.uniform(1))
        result = driver.find_element_by_id('resultStats').text.strip().replace('.','')
        result = int(str([int(s) for s in result if s.isdigit()]).strip('[]').replace(',','').replace(' ',''))
        df.loc[i, 'as_google_18'] = result
        print((i,df.nome[i]), end=' ')
    
driver.quit()

In [ ]:
d = df[['ff_google_all', 'ff_google_18', 'as_google_all', 'as_google_18']]
d.to_csv('columns/ff_as_google.csv', index=False)